In [24]:
import pandas as pd
import cvxpy as cp
import numpy as  np
from mamimo.time_utils import add_time_features, PowerTrend
from mamimo.carryover import ExponentialCarryover
from mamimo.saturation import ExponentialSaturation
import pandas as pd
import numpy as np

from mamimo.time_utils import add_time_features, PowerTrend
from mamimo.carryover import ExponentialCarryover
from mamimo.saturation import ExponentialSaturation

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin, OneToOneFeatureMixin

from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

import holidays

import seaborn as sns
import matplotlib.pyplot as plt


In [7]:

class AddWeekendsTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        # This transformer doesn't need to learn any parameters during fitting,
        # so we simply return self.
        return self

    def transform(self, X):
        """Creates a new column with row value = 1 if the day is a Friday or Saturday and 0 if not."""
        df = X.copy()  # Create a copy of the input DataFrame to avoid modifying it directly
        weekday_values = df.index.weekday
        df['fri_sat'] = ((weekday_values == 4) | (weekday_values == 5)).astype(int)
        return df[['fri_sat']]

In [8]:
def minmax_fb(x):
    scaled_fb = ((x - 0)/(2294.69 - 0))
    return scaled_fb
def minmax_gg(x):
    scaled_gg = ((x - 0)/(480.66 - 0))
    return scaled_gg
def minmax_tt(x):
    scaled_tt = ((x - 0)/(840.37- 0))
    return scaled_tt


In [9]:
intercept= 133.12031243
weekend= -69.17339136
facebook= 1236.51148254
google= 280
tiktok= 1782.03178883

In [12]:
minmax_fb(120)

0.05229464546409319

In [50]:
def find_optimal_budget(TOTAL_DAILY_BUDGET, Date):
    intercept= 133.12031243
    weekend= -69.17339136
    facebook= 1236.51148254
    google= 280
    tiktok= 1782.03178883
    df=pd.DataFrame([pd.to_datetime(Date)])
    df.set_index(0, inplace=True)
    df['day']=Date
    df['day']=pd.to_datetime(df['day'])

    #treansforming the date df

    transformed_df = AddWeekendsTransformer().fit_transform(df)
    transformed_df.columns = ['weekends']


    # model cvxpy

    # Variables
    facebook_var = cp.Variable(pos=True)
    google_var   = cp.Variable(pos=True)
    tiktok_var  = cp.Variable(pos=True)
    breakpoint()

    # Constraint
    constraint = [google_var + facebook_var + tiktok_var <= TOTAL_DAILY_BUDGET, google_var>=TOTAL_DAILY_BUDGET*0.04769543069031596, facebook_var>=TOTAL_DAILY_BUDGET*0.16612285992471154, tiktok_var>=TOTAL_DAILY_BUDGET*0.0361817093849725]


    # Objective
    obj= cp.Maximize(intercept+ weekend* transformed_df['weekends'][0]+facebook *minmax_fb(facebook_var) + google *minmax_gg(google_var)+ tiktok*minmax_tt(tiktok_var))
    # from IPython.core.debugger import Pdb; Pdb().set_trace()
    # Solve
    prob = cp.Problem(obj, constraint)
    prob.solve(solver='OSQP', verbose=False)
    # Print solution
    status=prob.status
    value_Total_sales=prob.value
    value_google_spend=round(google_var.value)
    value_facebook_spend=round(facebook_var.value)
    value_tiktok_spend=round(tiktok_var.value)

    return status, value_Total_sales, value_google_spend, value_facebook_spend, value_tiktok_spend

In [51]:
find_optimal_budget(1000, '2023-09-08')

('optimal', 1848.3714068540285, 48, 166, 786)

In [38]:
minmax_gg(100)

0.20804726833936668

In [40]:
Date = '2023-09-08'

In [41]:
    df=pd.DataFrame([pd.to_datetime(Date)])
    df.set_index(0, inplace=True)
    df['day']=Date
    df['day']=pd.to_datetime(df['day'])


In [42]:
df

,day
0,
2023-09-08,2023-09-08


In [44]:
transformed_df = AddWeekendsTransformer().fit_transform(df)
transformed_df

,fri_sat
0,
2023-09-08,1


In [46]:
transformed_df.columns = ['weekends']



In [47]:
transformed_df

,weekends
0,
2023-09-08,1


In [66]:
intercept+ (weekend*transformed_df['weekends'][0])+ (facebook *minmax_fb(166)) + (google *minmax_gg(44))+ (tiktok*minmax_tt(786))

1845.7671742695752

In [121]:
def query_click_predict(date: str, facebook: float, google: float, tiktok: float):

    """
    Returns predicted sales data from "df" according input data
    """
    # pipeline= load_pipeline()

    # creating 1 row data as input, X_pred
    intercept, weekend, facebook, google, tiktok = (133.12031243, -69.17339136, 1236.51148254, 280, 1782.03178883)

    b = np.array([intercept, weekend, facebook, google, tiktok])

    date = pd.to_datetime(date)
    weekend_date = int((date.day_of_week==4) | (date.day_of_week==5))

    dict_feat = {
        "date": weekend_date,
        "facebook": minmax_fb(facebook),
        "google": minmax_gg(google),
        "tiktok": minmax_tt(tiktok)
    }

    X = np.array(list(dict_feat.values()))
    res = b[0] + np.dot(X, b[1:])

    return res

In [123]:
intercept, weekend, facebook, google, tiktok = (133.12031243, -69.17339136, 1236.51148254, 280, 1782.03178883)

b = np.array([intercept, weekend, facebook, google, tiktok])


In [124]:
b

array([ 133.12031243,  -69.17339136, 1236.51148254,  280.        ,
       1782.03178883])

In [122]:
query_click_predict('2023-09-08', 166, 44, 786)

4672.215865387753

In [65]:
facebook *minmax_fb(166)

89.45038593519823

In [70]:
intercept, weekend, facebook, google, tiktok

(133.12031243, -69.17339136, 1236.51148254, 280, 1782.03178883)

In [91]:
day = transformed_df['weekends'][0]
fbcost = 166
ggcost = 44
ttcost = 786

In [76]:
minmax_fb(fbcost)

0.07234092622532891

In [77]:
intercept+ (weekend*transformed_df['weekends'][0])+ (facebook *minmax_fb(166)) + (google *minmax_gg(44))+ (tiktok*minmax_tt(786))

1845.7671742695752

In [86]:
date = pd.to_datetime('2023-09-08')
weekend_date = int((date.day_of_week==4) | (date.day_of_week==5))


In [102]:
    dict_feat = {
        "date": weekend_date,
        "facebook": minmax_fb(fbcost),
        "google": minmax_gg(ggcost),
        "tiktok": minmax_tt(ttcost)
    }


In [103]:
dict_feat

{'date': 1,
 'facebook': 0.07234092622532891,
 'google': 0.09154079806932135,
 'tiktok': 0.9353023073170151}

In [104]:
print(minmax_fb(fbcost))
print(minmax_gg(ggcost))
print(minmax_tt(ttcost))

0.07234092622532891
0.09154079806932135
0.9353023073170151


In [105]:
np.array(list(dict_feat.values()))

array([1.        , 0.07234093, 0.0915408 , 0.93530231])

In [112]:
    X=np.array(list(dict_feat.values()))
X

array([1.        , 0.07234093, 0.0915408 , 0.93530231])

In [113]:
    b = np.array([intercept, weekend, facebook, google, tiktok])


In [114]:
b

array([ 133.12031243,  -69.17339136, 1236.51148254,  280.        ,
       1782.03178883])

In [117]:
np.dot(X, b[1:]) + b[0]

1845.7671742695752

In [118]:
b[0] + np.dot(X, b[1:])

1845.7671742695752

In [57]:
    b = np.array([intercept, weekend, facebook, google, tiktok])

In [58]:
day=transformed_df

array([ 133.12031243,  -69.17339136, 1236.51148254,  280.        ,
       1782.03178883])

In [ ]:
np.array(list(dict_feat.values()))

In [125]:
def newfun(date: str, facebook: float, google: float, tiktok: float):
    '''
    function
    '''
    b = np.array([133.12031243, -69.17339136, 1236.51148254, 280, 1782.03178883])
    date = pd.to_datetime(date)
    weekend_date = int((date.day_of_week==4) | (date.day_of_week==5))

    X = np.array([weekend_date, minmax_fb(facebook), minmax_gg(google), minmax_tt(tiktok)])
    
    res = b[0] + np.dot(X, b[1:])

    return res  

In [126]:
newfun('2023-09-08', 166, 44, 786)

1845.7671742695752